## Extracting Data from the Master Health Facility Registry (KMHFR)

In this exercise, we will use the following Python libraries:

- **Beautiful Soup**: For parsing HTML and XML documents.
- **Requests**: For making HTTP requests to retrieve data from the web.
- **JSON**: For handling JSON data structures.
- **Pandas**: For data manipulation and analysis.

### About the Master Health Facility Registry (KMHFR)

**Master Health Facility Registry (KMHFR)** is an application that contains information on all health facilities and community units in Kenya. Each health facility and community unit is identified with a unique code, along with details describing its geographical location, administrative location, ownership, type, and the services offered.

### Accessing the Data

The data from KMHFR can be accessed through an API, with documentation available [here](https://mfl-api-docs.readthedocs.io/en/latest/).

However, in this exercise, we will use **Beautiful Soup** to scrape the data from the KMHFR website. 


We'll be working with the following base URL:
```python
base_url = "https://kmhfr.health.go.ke/public/facilities?page={}"



**NB: Ethical Considerations of Web Scraping**

Web scraping is a powerful tool for extracting data from websites, but it comes with ethical and legal responsibilities. When scraping data from websites, it is important to:
- **Respect the website's robots.txt file**: This file specifies which parts of the site should not be accessed by automated tools. Ignoring this can lead to legal issues or your IP being blocked.
- **Avoid overloading the server**: Sending too many requests in a short period can strain the website's server, potentially causing disruptions in service.
- **Use data responsibly**: Ensure that the data you scrape is used in accordance with the terms of service of the website, and respect privacy and copyright laws.


In this exercise, web scraping is used solely for demonstration purposes. The goal is to illustrate how web scraping works and how data can be extracted and processed using Python libraries. In practice, it is recommended to use official APIs when available, as they are designed to provide access to data in a controlled and ethical manner.

### Import necessary Libraries

In [3]:
import requests  # Used for making HTTP requests to fetch data from websites or APIs
from bs4 import BeautifulSoup as bs  # BeautifulSoup is used for parsing HTML and XML documents, enabling easy extraction of data from web pages
import pandas as pd  # Pandas is a data manipulation and analysis library that provides data structures like DataFrames for handling and analyzing tabular data
import json  # The JSON library is used for working with JSON data, allowing you to parse JSON strings or convert Python objects to JSON
import time  # The time library is used to add delays or measure the time taken by code to execute, which can be useful for pacing requests during web scraping
import sys  # The sys library allows interaction with the Python runtime environment, including manipulating output streams and command-line arguments
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

## Data Scraping

##### Get all the url pages

In [4]:
# Define the base URL for the KMHFR data. The placeholder `{}` will be replaced with page numbers.
base_url = "https://kmhfr.health.go.ke/public/facilities?page={}"

# Generate a list of URLs for each page from 1 to 494 by replacing the placeholder in the base URL with the page numbers.
urls = [base_url.format(page) for page in range(1, 495)]

# Print the list of generated URLs to verify the URLs created for each page.
print(urls)


['https://kmhfr.health.go.ke/public/facilities?page=1', 'https://kmhfr.health.go.ke/public/facilities?page=2', 'https://kmhfr.health.go.ke/public/facilities?page=3', 'https://kmhfr.health.go.ke/public/facilities?page=4', 'https://kmhfr.health.go.ke/public/facilities?page=5', 'https://kmhfr.health.go.ke/public/facilities?page=6', 'https://kmhfr.health.go.ke/public/facilities?page=7', 'https://kmhfr.health.go.ke/public/facilities?page=8', 'https://kmhfr.health.go.ke/public/facilities?page=9', 'https://kmhfr.health.go.ke/public/facilities?page=10', 'https://kmhfr.health.go.ke/public/facilities?page=11', 'https://kmhfr.health.go.ke/public/facilities?page=12', 'https://kmhfr.health.go.ke/public/facilities?page=13', 'https://kmhfr.health.go.ke/public/facilities?page=14', 'https://kmhfr.health.go.ke/public/facilities?page=15', 'https://kmhfr.health.go.ke/public/facilities?page=16', 'https://kmhfr.health.go.ke/public/facilities?page=17', 'https://kmhfr.health.go.ke/public/facilities?page=18', 

##### Fetch json data for each url and combine them to a single dataframe

In [ ]:
# Function to fetch JSON data
def fetch_json_data(url, headers):
    """
    Fetch JSON data from a given URL and headers.
    
    Parameters:
        url (str): The URL to fetch data from.
        headers (dict): The headers to use for the request.
        
    Returns:
        dict: The parsed JSON data.
    """
    # Send a GET request to the URL with the provided headers
    response = requests.get(url, headers=headers)
    
    # Parse the response content using BeautifulSoup
    soup = bs(response.text, "html.parser")
    
    # Find the script tag containing the JSON data
    script_tag = soup.find('script', id='__NEXT_DATA__')
    
    if script_tag:
        # Load and return the JSON data
        data = json.loads(script_tag.string)
        return data
    return None

# Function to convert JSON data to DataFrame
def json_to_dataframe(json_data):
    """
    Convert JSON data to a DataFrame.
    
    Parameters:
        json_data (dict): The JSON data to convert.
        
    Returns:
        DataFrame: A DataFrame containing the JSON data.
    """
    # Extract the results from the JSON data
    results = json_data.get('props', {}).get('pageProps', {}).get('data', {}).get('results', [])
    
    # Convert the results list to a DataFrame
    df = pd.DataFrame(results)
    return df

# Define the base URL for fetching data, with a placeholder for page numbers
base_url = "https://kmhfr.health.go.ke/public/facilities?page={}"

# Generate a list of URLs for each page from 1 to 494
urls = [base_url.format(page) for page in range(1, 495)]

# Define headers for HTTP requests to mimic a browser request
HEADERS = {
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9,fr;q=0.8",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}

# Initialize an empty DataFrame to accumulate all the fetched data
all_data = pd.DataFrame()

# Loop through each URL and fetch data
for url in urls:
    data = fetch_json_data(url, HEADERS)
    if data:
        # Convert JSON data to DataFrame
        df = json_to_dataframe(data)
        # Append the DataFrame to the accumulated data
        all_data = pd.concat([all_data, df], ignore_index=True)


In [ ]:
# Print the shape of the DataFrame (number of rows and columns)
print(f"DataFrame shape: {all_data.shape}")

# Print the number of rows in the DataFrame
print(f"Number of rows: {all_data.shape[0]}")

# Print the number of columns in the DataFrame
print(f"Number of columns: {all_data.shape[1]}")




DataFrame shape: (14805, 47)
Number of rows: 14805
Number of columns: 47

### Save dataframe to local dir

In [ ]:
# Save the combined DataFrame to a file
all_data.to_csv("data/all_facilities.csv", index=False)

In [ ]:
all_data.columns

In [ ]:
all_data.officialname.unique()

In [ ]:
null_counts = all_data.isnull().sum()
print("Null values per column:")
print(null_counts)

### Select Baringo County from the scraped data

In [8]:
county_names = df2['county_name'].value_counts()
county_names_list = county_names.index.tolist()

# Display the number of unique county names
print(f"\nNumber of unique county names: {len(county_names_list)}")




Number of unique county names: 47


In [9]:
baringo_df = df2[df2['county_name'] == 'Baringo']
baringo_df


,code,name,officialname,registration_number,keph_level_name,facility_type_name,facility_type_category,county,constituency,ward,...,infrastructure_categories,speciality_names,speciality_categories,is_public_visible,created,closed,is_published,id,lat,long
14,31993.0,Ngambo Health Care,Ngambo Health Care,NaN,Level 2,Medical Clinic,MEDICAL CLINIC,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,2e793f53-d493-485d-a6a1-efe5860c4c6c,...,[],"['Pharm Tech', 'Casuals', 'KRCHN', 'General Cl...","['GENERAL SUPPORT STAFFS', 'CLINICAL OFFICERS'...",True,2024-07-16T12:59:46.767058Z,False,True,d2dd346b-20a5-4262-8390-13dabf4cd2f1,35.982215,0.470296
16,31994.0,Mbechot Dispensary,Mbechot Dispensary,NaN,Level 2,Dispensary,DISPENSARY,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,6cd0e2df-f8da-4dc5-96a2-8517dde83168,...,[],"['KRCHN', 'Security']","['SUPPORT STAFF', 'NURSES AND SPECIALIST']",True,2024-07-16T12:15:18.577285Z,False,True,cfceebed-8a94-43aa-8421-a243f415b522,36.106182,0.371174
18,31986.0,Elicare Medical Clinic,Elicare Medical Clinic,020807,Level 2,Medical Clinic,MEDICAL CLINIC,7c942357-44a8-49c9-9ca6-8247ad903b57,8b79d994-5b2a-4686-9b89-85b5e6c709d6,2ccd3cc3-0f90-47d3-adb4-793fea3630d9,...,[],['General Clinical Officers(Diploma)'],['CLINICAL OFFICERS'],True,2024-07-16T07:16:53.114238Z,False,True,9701bcd2-008a-4137-a68a-9339370c5a6d,36.002876,0.068192
291,31598.0,Lokaimoi Medicare LTD,Lokaimoi Medicare LTD,COC/F/110,Level 2,Medical Center,MEDICAL CENTER,7c942357-44a8-49c9-9ca6-8247ad903b57,e9d5c0e1-a0c3-4053-bd2a-052ff54a64e3,e8b07b8b-7499-4a1a-8809-67758b7f8d19,...,[],"['Dental Technologists', 'Casuals', 'Cleaners'...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-02-26T10:46:19.937796Z,False,True,a76a2c4f-b92b-4614-8272-f2923fa13965,35.791240,0.619311
324,31366.0,Equity Afia Kabarnet Medical Center,Equity Afia Kabarnet Medical Center,020541,Level 3,Medical Center,MEDICAL CENTER,7c942357-44a8-49c9-9ca6-8247ad903b57,9fcb8e52-0034-4e8a-a610-c94c70cfa213,adff389b-2636-404b-a999-df218328df84,...,[],"['General Medical Officers', 'Medical Laborato...","['MEDICAL LABORATORY', 'MEDICAL OFFICERS & SPE...",True,2024-02-09T13:28:08.854025Z,False,True,cc82f357-31c1-487f-83a2-6aeab52df07e,35.719931,0.492734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14584,14867.0,Kimalel Health Centre,Kimalel Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,2e793f53-d493-485d-a6a1-efe5860c4c6c,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,7a8d3b0d-7d24-4a23-8034-6607859f3632,35.891400,0.464780
14643,15386.0,Ol-Arabel Dispensary,Ol-Arabel Dispensary,NaN,Level 2,Dispensary,DISPENSARY,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,6cd0e2df-f8da-4dc5-96a2-8517dde83168,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,681d56dc-7c5f-40b9-83eb-98722120e471,36.254160,0.275930
14667,15197.0,Mogorwa Health Centre,Mogorwa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,9fcb8e52-0034-4e8a-a610-c94c70cfa213,9eea54e2-7105-4f5f-bb9d-c15c02c06e88,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,2c21d8e8-a26a-4c68-9a03-904018e072ec,35.716670,0.308830
14674,14979.0,Kolowa Health Centre,Kolowa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,655c6da0-4600-4ae0-9682-ad7e5e3b1c65,2a957f58-cd65-4b08-bbb5-dd0834aaaff8,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,f7696e3f-63e7-44fc-b4ca-560c457a7a23,35.758700,1.216200


In [10]:
# Optionally, you can save the merged DataFrame to a new CSV file
baringo_df.to_csv('data/baringo_df.csv', index=False)

## Data Visualization

- **df1**: Baringo Health facilities.
- **df2**: Our data from KMHFR.

In [26]:
# Read the CSV files
df1 = pd.read_csv('data/BHF.csv') # data shared
# df2 = all_data
baringo_df = pd.read_csv('data/baringo_df.csv') #baringo df

In [27]:
# df2.shape[0]
baringo_df.columns

Index(['code', 'name', 'officialname', 'registration_number',
       'keph_level_name', 'facility_type_name', 'facility_type_category',
       'county', 'constituency', 'ward', 'owner_name', 'owner_type_name',
       'regulatory_body_name', 'beds', 'cots', 'search', 'county_name',
       'constituency_name', 'sub_county', 'sub_county_name', 'ward_name',
       'keph_level', 'facility_type', 'owner_type', 'owner',
       'operation_status', 'operation_status_name', 'admission_status_name',
       'open_whole_day', 'open_public_holidays', 'open_weekends',
       'open_late_night', 'services', 'categories', 'service_names',
       'infrastructure', 'infrastructure_names', 'infrastructure_categories',
       'speciality_names', 'speciality_categories', 'is_public_visible',
       'created', 'closed', 'is_published', 'id', 'lat', 'long'],
      dtype='object')

In [49]:
baringo_df

,code,name,officialname,registration_number,keph_level_name,facility_type_name,facility_type_category,county,constituency,ward,...,infrastructure_categories,speciality_names,speciality_categories,is_public_visible,created,closed,is_published,id,lat,long
0,31993.0,Ngambo Health Care,Ngambo Health Care,NaN,Level 2,Medical Clinic,MEDICAL CLINIC,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,2e793f53-d493-485d-a6a1-efe5860c4c6c,...,[],"['Pharm Tech', 'Casuals', 'KRCHN', 'General Cl...","['GENERAL SUPPORT STAFFS', 'CLINICAL OFFICERS'...",True,2024-07-16T12:59:46.767058Z,False,True,d2dd346b-20a5-4262-8390-13dabf4cd2f1,35.982215,0.470296
1,31994.0,Mbechot Dispensary,Mbechot Dispensary,NaN,Level 2,Dispensary,DISPENSARY,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,6cd0e2df-f8da-4dc5-96a2-8517dde83168,...,[],"['KRCHN', 'Security']","['SUPPORT STAFF', 'NURSES AND SPECIALIST']",True,2024-07-16T12:15:18.577285Z,False,True,cfceebed-8a94-43aa-8421-a243f415b522,36.106182,0.371174
2,31986.0,Elicare Medical Clinic,Elicare Medical Clinic,020807,Level 2,Medical Clinic,MEDICAL CLINIC,7c942357-44a8-49c9-9ca6-8247ad903b57,8b79d994-5b2a-4686-9b89-85b5e6c709d6,2ccd3cc3-0f90-47d3-adb4-793fea3630d9,...,[],['General Clinical Officers(Diploma)'],['CLINICAL OFFICERS'],True,2024-07-16T07:16:53.114238Z,False,True,9701bcd2-008a-4137-a68a-9339370c5a6d,36.002876,0.068192
3,31598.0,Lokaimoi Medicare LTD,Lokaimoi Medicare LTD,COC/F/110,Level 2,Medical Center,MEDICAL CENTER,7c942357-44a8-49c9-9ca6-8247ad903b57,e9d5c0e1-a0c3-4053-bd2a-052ff54a64e3,e8b07b8b-7499-4a1a-8809-67758b7f8d19,...,[],"['Dental Technologists', 'Casuals', 'Cleaners'...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-02-26T10:46:19.937796Z,False,True,a76a2c4f-b92b-4614-8272-f2923fa13965,35.791240,0.619311
4,31366.0,Equity Afia Kabarnet Medical Center,Equity Afia Kabarnet Medical Center,020541,Level 3,Medical Center,MEDICAL CENTER,7c942357-44a8-49c9-9ca6-8247ad903b57,9fcb8e52-0034-4e8a-a610-c94c70cfa213,adff389b-2636-404b-a999-df218328df84,...,[],"['General Medical Officers', 'Medical Laborato...","['MEDICAL LABORATORY', 'MEDICAL OFFICERS & SPE...",True,2024-02-09T13:28:08.854025Z,False,True,cc82f357-31c1-487f-83a2-6aeab52df07e,35.719931,0.492734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,14867.0,Kimalel Health Centre,Kimalel Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,2e793f53-d493-485d-a6a1-efe5860c4c6c,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,7a8d3b0d-7d24-4a23-8034-6607859f3632,35.891400,0.464780
301,15386.0,Ol-Arabel Dispensary,Ol-Arabel Dispensary,NaN,Level 2,Dispensary,DISPENSARY,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,6cd0e2df-f8da-4dc5-96a2-8517dde83168,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,681d56dc-7c5f-40b9-83eb-98722120e471,36.254160,0.275930
302,15197.0,Mogorwa Health Centre,Mogorwa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,9fcb8e52-0034-4e8a-a610-c94c70cfa213,9eea54e2-7105-4f5f-bb9d-c15c02c06e88,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,2c21d8e8-a26a-4c68-9a03-904018e072ec,35.716670,0.308830
303,14979.0,Kolowa Health Centre,Kolowa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,655c6da0-4600-4ae0-9682-ad7e5e3b1c65,2a957f58-cd65-4b08-bbb5-dd0834aaaff8,...,[],[],[],True,2009-10-31T00:00:00Z,False,True,f7696e3f-63e7-44fc-b4ca-560c457a7a23,35.758700,1.216200


### Convert to gdf

In [20]:
baringo_df.columns

Index(['code', 'name', 'officialname', 'registration_number',
       'keph_level_name', 'facility_type_name', 'facility_type_category',
       'county', 'constituency', 'ward', 'owner_name', 'owner_type_name',
       'regulatory_body_name', 'beds', 'cots', 'search', 'county_name',
       'constituency_name', 'sub_county', 'sub_county_name', 'ward_name',
       'keph_level', 'facility_type', 'owner_type', 'owner',
       'operation_status', 'operation_status_name', 'admission_status_name',
       'open_whole_day', 'open_public_holidays', 'open_weekends',
       'open_late_night', 'services', 'categories', 'service_names',
       'infrastructure', 'infrastructure_names', 'infrastructure_categories',
       'speciality_names', 'speciality_categories', 'is_public_visible',
       'created', 'closed', 'is_published', 'id', 'lat', 'long'],
      dtype='object')

In [37]:
# Create a GeoDataFrame by converting latitude and longitude to geometry points
geometry = [Point(xy) for xy in zip(baringo_df['lat'], baringo_df['long'])]
gdf = gpd.GeoDataFrame(baringo_df, geometry=geometry)

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

,code,name,officialname,registration_number,keph_level_name,facility_type_name,facility_type_category,county,constituency,ward,...,speciality_names,speciality_categories,is_public_visible,created,closed,is_published,id,lat,long,geometry
0,31993.0,Ngambo Health Care,Ngambo Health Care,NaN,Level 2,Medical Clinic,MEDICAL CLINIC,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,2e793f53-d493-485d-a6a1-efe5860c4c6c,...,"['Pharm Tech', 'Casuals', 'KRCHN', 'General Cl...","['GENERAL SUPPORT STAFFS', 'CLINICAL OFFICERS'...",True,2024-07-16T12:59:46.767058Z,False,True,d2dd346b-20a5-4262-8390-13dabf4cd2f1,35.982215,0.470296,POINT (35.98221 0.47030)
1,31994.0,Mbechot Dispensary,Mbechot Dispensary,NaN,Level 2,Dispensary,DISPENSARY,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,6cd0e2df-f8da-4dc5-96a2-8517dde83168,...,"['KRCHN', 'Security']","['SUPPORT STAFF', 'NURSES AND SPECIALIST']",True,2024-07-16T12:15:18.577285Z,False,True,cfceebed-8a94-43aa-8421-a243f415b522,36.106182,0.371174,POINT (36.10618 0.37117)
2,31986.0,Elicare Medical Clinic,Elicare Medical Clinic,020807,Level 2,Medical Clinic,MEDICAL CLINIC,7c942357-44a8-49c9-9ca6-8247ad903b57,8b79d994-5b2a-4686-9b89-85b5e6c709d6,2ccd3cc3-0f90-47d3-adb4-793fea3630d9,...,['General Clinical Officers(Diploma)'],['CLINICAL OFFICERS'],True,2024-07-16T07:16:53.114238Z,False,True,9701bcd2-008a-4137-a68a-9339370c5a6d,36.002876,0.068192,POINT (36.00288 0.06819)
3,31598.0,Lokaimoi Medicare LTD,Lokaimoi Medicare LTD,COC/F/110,Level 2,Medical Center,MEDICAL CENTER,7c942357-44a8-49c9-9ca6-8247ad903b57,e9d5c0e1-a0c3-4053-bd2a-052ff54a64e3,e8b07b8b-7499-4a1a-8809-67758b7f8d19,...,"['Dental Technologists', 'Casuals', 'Cleaners'...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-02-26T10:46:19.937796Z,False,True,a76a2c4f-b92b-4614-8272-f2923fa13965,35.791240,0.619311,POINT (35.79124 0.61931)
4,31366.0,Equity Afia Kabarnet Medical Center,Equity Afia Kabarnet Medical Center,020541,Level 3,Medical Center,MEDICAL CENTER,7c942357-44a8-49c9-9ca6-8247ad903b57,9fcb8e52-0034-4e8a-a610-c94c70cfa213,adff389b-2636-404b-a999-df218328df84,...,"['General Medical Officers', 'Medical Laborato...","['MEDICAL LABORATORY', 'MEDICAL OFFICERS & SPE...",True,2024-02-09T13:28:08.854025Z,False,True,cc82f357-31c1-487f-83a2-6aeab52df07e,35.719931,0.492734,POINT (35.71993 0.49273)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,14867.0,Kimalel Health Centre,Kimalel Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,2e793f53-d493-485d-a6a1-efe5860c4c6c,...,[],[],True,2009-10-31T00:00:00Z,False,True,7a8d3b0d-7d24-4a23-8034-6607859f3632,35.891400,0.464780,POINT (35.89140 0.46478)
301,15386.0,Ol-Arabel Dispensary,Ol-Arabel Dispensary,NaN,Level 2,Dispensary,DISPENSARY,7c942357-44a8-49c9-9ca6-8247ad903b57,ca41dc5d-5673-4344-96b8-b4fde34b93bc,6cd0e2df-f8da-4dc5-96a2-8517dde83168,...,[],[],True,2009-10-31T00:00:00Z,False,True,681d56dc-7c5f-40b9-83eb-98722120e471,36.254160,0.275930,POINT (36.25416 0.27593)
302,15197.0,Mogorwa Health Centre,Mogorwa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,9fcb8e52-0034-4e8a-a610-c94c70cfa213,9eea54e2-7105-4f5f-bb9d-c15c02c06e88,...,[],[],True,2009-10-31T00:00:00Z,False,True,2c21d8e8-a26a-4c68-9a03-904018e072ec,35.716670,0.308830,POINT (35.71667 0.30883)
303,14979.0,Kolowa Health Centre,Kolowa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,7c942357-44a8-49c9-9ca6-8247ad903b57,655c6da0-4600-4ae0-9682-ad7e5e3b1c65,2a957f58-cd65-4b08-bbb5-dd0834aaaff8,...,[],[],True,2009-10-31T00:00:00Z,False,True,f7696e3f-63e7-44fc-b4ca-560c457a7a23,35.758700,1.216200,POINT (35.75870 1.21620)


In [34]:
gdf.columns

Index(['code', 'name', 'officialname', 'registration_number',
       'keph_level_name', 'facility_type_name', 'facility_type_category',
       'county', 'constituency', 'ward', 'owner_name', 'owner_type_name',
       'regulatory_body_name', 'beds', 'cots', 'search', 'county_name',
       'constituency_name', 'sub_county', 'sub_county_name', 'ward_name',
       'keph_level', 'facility_type', 'owner_type', 'owner',
       'operation_status', 'operation_status_name', 'admission_status_name',
       'open_whole_day', 'open_public_holidays', 'open_weekends',
       'open_late_night', 'services', 'categories', 'service_names',
       'infrastructure', 'infrastructure_names', 'infrastructure_categories',
       'speciality_names', 'speciality_categories', 'is_public_visible',
       'created', 'closed', 'is_published', 'id', 'lat', 'long', 'geometry'],
      dtype='object')

In [46]:
# Print the number of rows in the DataFrame
print(f"Number of rows: {gdf.shape[0]}")
gdf.explore(
    column='facility_type_category',  # Column to color by
    cmap='viridis',            # Colormap
    legend=True,               # Show legend
    legend_kwds={
        'loc': 'topright',     # Position of the legend
        'title': 'Keph facility_type_category', # Title of the legend
    },
    tooltip='keph_level_name', # Tooltip information
    style_kwds=dict(radius=4, fillOpacity=0.7)  # Style options
)


Number of rows: 305


In [47]:
gdf.columns

Index(['code', 'name', 'officialname', 'registration_number',
       'keph_level_name', 'facility_type_name', 'facility_type_category',
       'county', 'constituency', 'ward', 'owner_name', 'owner_type_name',
       'regulatory_body_name', 'beds', 'cots', 'search', 'county_name',
       'constituency_name', 'sub_county', 'sub_county_name', 'ward_name',
       'keph_level', 'facility_type', 'owner_type', 'owner',
       'operation_status', 'operation_status_name', 'admission_status_name',
       'open_whole_day', 'open_public_holidays', 'open_weekends',
       'open_late_night', 'services', 'categories', 'service_names',
       'infrastructure', 'infrastructure_names', 'infrastructure_categories',
       'speciality_names', 'speciality_categories', 'is_public_visible',
       'created', 'closed', 'is_published', 'id', 'lat', 'long', 'geometry'],
      dtype='object')

###  drop unneccesary columns

In [53]:
# Drop specified columns from the GeoDataFrame
#gdf = gdf.drop(columns=['county', 'constituency', 'ward'])
gdf

,code,name,officialname,registration_number,keph_level_name,facility_type_name,facility_type_category,owner_name,owner_type_name,regulatory_body_name,...,speciality_names,speciality_categories,is_public_visible,created,closed,is_published,id,lat,long,geometry
0,31993.0,Ngambo Health Care,Ngambo Health Care,NaN,Level 2,Medical Clinic,MEDICAL CLINIC,Private Practice- Pharmacist,Private Practice,NaN,...,"['Pharm Tech', 'Casuals', 'KRCHN', 'General Cl...","['GENERAL SUPPORT STAFFS', 'CLINICAL OFFICERS'...",True,2024-07-16T12:59:46.767058Z,False,True,d2dd346b-20a5-4262-8390-13dabf4cd2f1,35.982215,0.470296,POINT (35.98221 0.47030)
1,31994.0,Mbechot Dispensary,Mbechot Dispensary,NaN,Level 2,Dispensary,DISPENSARY,Ministry of Health,Ministry of Health,Ministry of Health,...,"['KRCHN', 'Security']","['SUPPORT STAFF', 'NURSES AND SPECIALIST']",True,2024-07-16T12:15:18.577285Z,False,True,cfceebed-8a94-43aa-8421-a243f415b522,36.106182,0.371174,POINT (36.10618 0.37117)
2,31986.0,Elicare Medical Clinic,Elicare Medical Clinic,020807,Level 2,Medical Clinic,MEDICAL CLINIC,Private Practice - Clinical Officer,Private Practice,Clinical Officers Council,...,['General Clinical Officers(Diploma)'],['CLINICAL OFFICERS'],True,2024-07-16T07:16:53.114238Z,False,True,9701bcd2-008a-4137-a68a-9339370c5a6d,36.002876,0.068192,POINT (36.00288 0.06819)
3,31598.0,Lokaimoi Medicare LTD,Lokaimoi Medicare LTD,COC/F/110,Level 2,Medical Center,MEDICAL CENTER,Private Practice - Private Company,Private Practice,Clinical Officers Council,...,"['Dental Technologists', 'Casuals', 'Cleaners'...","['GENERAL SUPPORT STAFFS', 'MEDICAL LABORATORY...",True,2024-02-26T10:46:19.937796Z,False,True,a76a2c4f-b92b-4614-8272-f2923fa13965,35.791240,0.619311,POINT (35.79124 0.61931)
4,31366.0,Equity Afia Kabarnet Medical Center,Equity Afia Kabarnet Medical Center,020541,Level 3,Medical Center,MEDICAL CENTER,Private Practice - Private Company,Private Practice,Pharmacy & Poisons Board,...,"['General Medical Officers', 'Medical Laborato...","['MEDICAL LABORATORY', 'MEDICAL OFFICERS & SPE...",True,2024-02-09T13:28:08.854025Z,False,True,cc82f357-31c1-487f-83a2-6aeab52df07e,35.719931,0.492734,POINT (35.71993 0.49273)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,14867.0,Kimalel Health Centre,Kimalel Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,Ministry of Health,Ministry of Health,Ministry of Health,...,[],[],True,2009-10-31T00:00:00Z,False,True,7a8d3b0d-7d24-4a23-8034-6607859f3632,35.891400,0.464780,POINT (35.89140 0.46478)
301,15386.0,Ol-Arabel Dispensary,Ol-Arabel Dispensary,NaN,Level 2,Dispensary,DISPENSARY,Ministry of Health,Ministry of Health,Ministry of Health,...,[],[],True,2009-10-31T00:00:00Z,False,True,681d56dc-7c5f-40b9-83eb-98722120e471,36.254160,0.275930,POINT (36.25416 0.27593)
302,15197.0,Mogorwa Health Centre,Mogorwa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,Ministry of Health,Ministry of Health,Ministry of Health,...,[],[],True,2009-10-31T00:00:00Z,False,True,2c21d8e8-a26a-4c68-9a03-904018e072ec,35.716670,0.308830,POINT (35.71667 0.30883)
303,14979.0,Kolowa Health Centre,Kolowa Health Centre,NaN,Level 3,Basic Health Centre,HEALTH CENTRE,Ministry of Health,Ministry of Health,Ministry of Health,...,[],[],True,2009-10-31T00:00:00Z,False,True,f7696e3f-63e7-44fc-b4ca-560c457a7a23,35.758700,1.216200,POINT (35.75870 1.21620)


In [54]:
# Define the list of columns you want to display in the tooltip
tooltip_columns = ['name',  'beds', 'cots', 'search', 
       'operation_status', 'operation_status_name', 'admission_status_name',
       'open_whole_day', 'open_public_holidays', 'open_weekends',
       'open_late_night'] 

# Generate the interactive map with detailed tooltip
m = gdf.explore(
    column='facility_type_category',  # Column to color by
    cmap='viridis',            # Colormap
    legend=True,               # Show legend
    tooltip=tooltip_columns,   # List of columns to show in the tooltip
    style_kwds=dict(radius=4, fillOpacity=0.7)  # Style options
)

# Save the map as an HTML file
m.save('interactive_map.html')